In [ ]:
import sys
import os

sys.path.insert(0, os.path.abspath("."))
sys.path.insert(0, os.path.abspath("."))
sys.path.append(os.path.abspath("../../"))

os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.99"
# os.environ["XLA_PYTHON_CLIENT_ALLOCATOR"] = "platform"
# os.environ["XLA_FLAGS"] = (
#     "--xla_disable_hlo_passes=constant_folding"  # this disables constant folding
# )
from desc import set_device
set_device("gpu")

In [ ]:
import numpy as np
np.set_printoptions(linewidth=np.inf, precision=4, suppress=True, threshold=sys.maxsize)
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.graph_objects as go
import functools
import scipy

In [ ]:
import desc

from desc.basis import *
from desc.backend import *
from desc.compute import *
from desc.coils import *
from desc.equilibrium import *
from desc.examples import *
from desc.grid import *
from desc.geometry import *

from desc.objectives import *
from desc.objectives.objective_funs import *
from desc.objectives.getters import *
from desc.objectives.normalization import compute_scaling_factors
from desc.objectives.utils import *
from desc.optimize._constraint_wrappers import *

from desc.transform import Transform
from desc.plotting import *
from desc.optimize import *
from desc.perturbations import *
from desc.profiles import *
from desc.compat import *
from desc.utils import *
from desc.magnetic_fields import *

from desc.__main__ import main
from desc.vmec_utils import vmec_boundary_subspace
from desc.input_reader import InputReader
from desc.continuation import solve_continuation_automatic
from desc.compute.data_index import register_compute_fun
from desc.optimize.utils import solve_triangular_regularized

print_backend_info()

In [ ]:
from diffrax import *
from desc.particles import *
from desc.particles import _trace_particles
import timeit

In [ ]:
eqi = get("precise_QA")
eq = rescale(eq=eqi, L=("a", 1.7044), B=("<B>", 5.86), copy=True)

model = VacuumGuidingCenterTrajectory(frame="flux")

In [ ]:
# these will be used as 1e-T
ts_to_profile = [1, 2, 3, 4, 5]

In [ ]:
Ns = [1, 10, 30, 100, 300, 500, 1000, 3000, 5000, 10000]
Ns = np.array(Ns)
np.savetxt("Ns_used4all.txt", Ns)

for T in ts_to_profile:
    ts = np.linspace(0, 10**(-T), 100)

    @jit
    def fun(x0, args):
        rpz, _ = _trace_particles(
            field=eq,
            y0=x0,
            model_args=args,
            model=model,
            ts=ts,
        )
        return rpz

    Ts = []
    repeat = 5
    for n in Ns:
        rhos = [0.7] * n
        initializer = ManualParticleInitializerFlux(
            rho0=rhos,
            theta0=0,
            zeta0=np.random.rand(n) * 2 * np.pi,
            xi0=0.7,
            E=3.5e6,
            m=4.0,
            q=1.0,
        )
        x0, args = initializer.init_particles(model=model, field=eq)
        _ = fun(x0, args).block_until_ready()  # compile
        fun_time = lambda: fun(x0, args).block_until_ready()
        t = timeit.timeit(fun_time, number=repeat)
        print(f"N={n:^7} and Tf=1e-{T} took {t/repeat:.4f} seconds per run")
        Ts.append(t / repeat)

    Ts = np.array(Ts)
    np.savetxt(f"ts_1e-{T}.txt", Ts)

In [ ]:
Ns = np.loadtxt("Ns_used4all.txt")
for T in ts_to_profile:
    Tsi = np.loadtxt(f"ts_1e-{T}-qa.txt")
    plt.semilogx(Ns, Tsi, label=f"tf = 1e-{T}")
plt.xlabel("Number of particles")
plt.ylabel("Time per run (seconds)")
plt.legend()
plt.title("Time to trace particles in reactor size precise_QA")
plt.ylim([0, 10])
plt.savefig("time_to_trace_particles_precise_qa.png", dpi=500)

# Precise QH

In [ ]:
eqi = get("precise_QH")
eq = rescale(eq=eqi, L=("a", 1.7044), B=("<B>", 5.86), copy=True)

model = VacuumGuidingCenterTrajectory(frame="flux")
# these will be used as 1e-T
ts_to_profile = [6, 5, 4, 3]
Ns = [1, 10, 30, 100, 300, 500, 1000, 3000, 5000, 10000]
Ns = np.array(Ns)
np.savetxt("Ns_used4all-qh.txt", Ns)

In [ ]:
for T in ts_to_profile:
    ts = np.linspace(0, 10 ** (-T), 100)

    @jit
    def fun(x0, args):
        rpz, _ = _trace_particles(
            field=eq, y0=x0, model_args=args, model=model, ts=ts, min_step_size=1e-11,
        )
        return rpz

    Ts = []
    repeat = 5
    for n in Ns:
        rhos = [0.3] * n
        initializer = ManualParticleInitializerFlux(
            rho0=rhos,
            theta0=0,
            zeta0=np.random.rand(n) * 2 * np.pi,
            xi0=0.3,
            E=3.5e6,
            m=4.0,
            q=1.0,
        )
        x0, args = initializer.init_particles(model=model, field=eq)
        _ = fun(x0, args).block_until_ready()  # compile
        fun_time = lambda: fun(x0, args).block_until_ready()
        t = timeit.timeit(fun_time, number=repeat)
        print(f"N={n:^7} and Tf=1e-{T} took {t/repeat:.4f} seconds per run")
        Ts.append(t / repeat)

    Ts = np.array(Ts)
    np.savetxt(f"ts_1e-{T}-qh-same.txt", Ts)

In [ ]:
Ts = np.array(
    [7.7308, 75.0176, 79.4468, 78.9849, 82.2073, 93.2229, 112.6139, 136.8404, 80, 80]
)
np.savetxt(f"ts_1e-3-qh.txt", Ts)

In [ ]:
Ns = np.loadtxt("Ns_used4all-qh.txt")
ts_to_profile = [6, 5, 4]
for T in ts_to_profile:
    Tsi = np.loadtxt(f"ts_1e-{T}-qh-same.txt")
    plt.semilogx(Ns, Tsi, label=f"tf = 1e-{T}")
plt.xlabel("Number of particles")
plt.ylabel("Time per run (seconds)")
plt.legend()
plt.title("Time to trace particles in reactor size precise_QH")
# plt.ylim([0, 90])
plt.savefig("time_to_trace_particles_precise_qh_same.png", dpi=500)